In [88]:
import pandas as pd
import numpy as np

In [109]:
# ---------------------------
# Step 1: Load and clean the data
# ---------------------------
ratings_df = pd.read_csv("raw_data/ratings_v1.csv")
audio_snippets_df = pd.read_csv("raw_data/audio_snippets.csv")

# Display heads and lengths (for debugging purposes)
print(ratings_df.head())
print(audio_snippets_df.head())
print("Initial lengths:", len(ratings_df), len(audio_snippets_df))

# Drop rows where snippet_id is null
ratings_df = ratings_df.dropna(subset=["snippet_id"])
print("After dropping null snippet_id:", len(ratings_df))

# Drop duplicate ratings for the same snippet and prolific_id
ratings_df = ratings_df.drop_duplicates(subset=["snippet_id", "prolific_id"])
print("After dropping duplicates:", len(ratings_df))

                                     id               prolific_id  \
0  0944b194-9e71-4ab5-9438-c7ab3f326c9d  65cd505d9ba7450b9be3d6f7   
1  db01fe1c-30ac-422d-b4f9-81e5f2532d11  65cd505d9ba7450b9be3d6f7   
2  8da13785-e98e-4a20-a5a7-fe604a5d9a1b  65cd505d9ba7450b9be3d6f7   
3  0a7c3bf1-be52-4d2a-ad66-fb6137da25c7  65cd505d9ba7450b9be3d6f7   
4  98bf782a-a675-4b2e-afe8-5977b311be3e  65cd505d9ba7450b9be3d6f7   

                             snippet_id  aggression  frustration  annoyance  \
0  0e60f04a-575f-41a6-8449-6d8b3d1fab6f           3            7          7   
1  06a012e5-95c2-4ec8-9435-693b851a995b           2            8          7   
2  43081c99-a9ba-4b0a-8f8e-73a443409de4           1            1          1   
3  7d9a30f9-b288-4a6d-a3d4-302c4910c403           1            1          1   
4  e5633971-506b-4fc7-9fd2-99b848bdc0be           1            3          4   

                   created_at  
0  2025-03-18 14:54:42.046667  
1  2025-03-18 14:54:54.357149  
2  2025-03-18 

In [110]:
# ---------------------------
# Step 2: Compute per-snippet statistics and normalized agreement
# ---------------------------
emotions = ["aggression", "frustration", "annoyance"]

# Calculate mean, standard deviation, and count for each emotion by snippet
agg_functions = {emotion: ["mean", "std", "count"] for emotion in emotions}
snippet_stats = ratings_df.groupby("snippet_id").agg(agg_functions)

# Flatten MultiIndex column names (e.g., "aggression_mean", "aggression_std", "aggression_count")
snippet_stats.columns = ["_".join(col).strip() for col in snippet_stats.columns.values]
snippet_stats.reset_index(inplace=True)

# For the normalized agreement, assume ratings range from 1 to 10.
min_rating, max_rating = 1, 10
max_std = np.std([min_rating, max_rating], ddof=1)  # worst-case disagreement

# Compute normalized agreement for each emotion (1 = perfect, 0 = worst)
for emotion in emotions:
    std_col = f"{emotion}_std"
    agreement_col = f"{emotion}_agreement"
    # If std is NaN (e.g. only one rating), the agreement will be left as NaN.
    snippet_stats[agreement_col] = 1 - (snippet_stats[std_col] / max_std)

# (Optional) View the snippet-level stats
print("Snippet-level statistics:")
print(snippet_stats.head())

Snippet-level statistics:
                             snippet_id  aggression_mean  aggression_std  \
0  0016c3f7-2a20-4664-bc0e-0a4625a0c782             2.00        0.816497   
1  00f347f0-a283-4836-8939-cfc2501a00ba             2.50        1.732051   
2  0166bfb3-1548-44bd-92fd-142acb505d80             3.00        3.464102   
3  029bb33b-0bf1-4b49-8532-6552ec8eeada             6.00        3.559026   
4  047a7f10-73b0-4328-8cbe-78babd70c111             4.75        2.500000   

   aggression_count  frustration_mean  frustration_std  frustration_count  \
0                 4               4.0         2.160247                  4   
1                 4               7.0         1.414214                  4   
2                 3               4.0         3.464102                  3   
3                 4               7.0         4.082483                  4   
4                 4               8.5         1.290994                  4   

   annoyance_mean  annoyance_std  annoyance_count  agg

In [111]:
# ---------------------------
# Step 3: Define a function to drop the lowest 10% based on agreement
# ---------------------------
def drop_low_agreement(df, emotion, percentile=10):
    """
    Drops the lowest <percentile>% of snippets based on the normalized agreement for the given emotion.

    Parameters:
      df: DataFrame that contains snippet-level stats including the '{emotion}_agreement' column.
      emotion: The emotion for which to filter (e.g., 'aggression').
      percentile: The cutoff percentile (default is 10, meaning drop bottom 10%).

    Returns:
      Filtered DataFrame with only snippets above the cutoff.
    """
    agreement_col = f"{emotion}_agreement"
    # Calculate the cutoff value (lowest 10% agreement)
    cutoff = np.percentile(df[agreement_col].dropna(), percentile)
    # Keep only snippets with agreement above the cutoff
    filtered_df = df[df[agreement_col] > cutoff].copy()
    return filtered_df

In [113]:
# ---------------------------
# Step 4: Create individual emotion DataFrames, merge with audio snippets, and store results
# ---------------------------
emotion_dfs = {}
for emotion in emotions:
    # Filter out snippets with very low agreement for this emotion
    filtered_stats = drop_low_agreement(snippet_stats.copy(), emotion, percentile=10)

    # Create a temporary DataFrame with snippet_id, mean rating, and agreement for this emotion
    temp_df = filtered_stats[
        ["snippet_id", f"{emotion}_mean", f"{emotion}_agreement"]
    ].copy()
    temp_df.rename(
        columns={f"{emotion}_mean": "rating", f"{emotion}_agreement": "agreement"},
        inplace=True,
    )

    # Merge with the audio snippets DataFrame.
    # Note: ratings_df uses 'snippet_id' and audio_snippets_df uses 'id' for the snippet identifier.
    merged_df = pd.merge(
        temp_df, audio_snippets_df, left_on="snippet_id", right_on="id", how="inner"
    )

    # Retain only the file_path from audio_snippets and the average rating.
    final_df = merged_df[["file_path", "rating"]].copy()
    emotion_dfs[emotion] = final_df

    # Save the final DataFrame for this emotion to a CSV file.
    output_filename = f"ratings/{emotion}_ratings.csv"
    final_df.to_csv(output_filename, index=False)
    print(f"Saved {emotion} ratings to {output_filename}")

# (Optional) Print a preview of the final DataFrames
for emotion, df in emotion_dfs.items():
    print(f"\nEmotion: {emotion}")
    print(len(df))
    print(df.head())

Saved aggression ratings to data/aggression_ratings.csv
Saved frustration ratings to data/frustration_ratings.csv
Saved annoyance ratings to data/annoyance_ratings.csv

Emotion: aggression
                        file_path  rating
0   actor1_call_10_sentence_9.wav    2.00
1  actor1_call_14_sentence_22.wav    2.50
2    actor1_call_9_sentence_6.wav    3.00
3    actor1_call_7_sentence_5.wav    6.00
4  actor1_call_15_sentence_15.wav    4.75

Emotion: frustration
                        file_path  rating
0   actor1_call_10_sentence_9.wav     4.0
1  actor1_call_14_sentence_22.wav     7.0
2    actor1_call_9_sentence_6.wav     4.0
3  actor1_call_15_sentence_15.wav     8.5
4   actor1_call_14_sentence_1.wav     2.0

Emotion: annoyance
                        file_path  rating
0   actor1_call_10_sentence_9.wav     4.5
1  actor1_call_14_sentence_22.wav     5.5
2    actor1_call_9_sentence_6.wav     4.0
3  actor1_call_15_sentence_15.wav     8.0
4   actor1_call_14_sentence_1.wav     2.0


In [116]:
# ---------------------------
# Step 5: Create binary classification DataFrames per emotion
# ---------------------------
binary_emotion_dfs = {}
for emotion, df in emotion_dfs.items():
    # Create a copy and binarize the rating (ratings > 5 become 1, otherwise 0)
    df_binary = df.copy()
    df_binary["binary_rating"] = (df_binary["rating"] > 5).astype(int)
    # Drop the original ordinal rating, keeping only file_path and binary_rating
    df_binary = df_binary[["file_path", "binary_rating"]]
    binary_emotion_dfs[emotion] = df_binary

    # Save the binary DataFrame for this emotion
    binary_output_filename = f"ratings/{emotion}_binary_ratings.csv"
    df_binary.to_csv(binary_output_filename, index=False)
    print(f"Saved {emotion} binary ratings to {binary_output_filename}")

# Optional: Preview the binary DataFrame for one emotion (e.g., aggression)
print("\nPreview of binary ratings for 'aggression':")
print(binary_emotion_dfs["aggression"].head())

Saved aggression binary ratings to data/aggression_binary_ratings.csv
Saved frustration binary ratings to data/frustration_binary_ratings.csv
Saved annoyance binary ratings to data/annoyance_binary_ratings.csv

Preview of binary ratings for 'aggression':
                        file_path  binary_rating
0   actor1_call_10_sentence_9.wav              0
1  actor1_call_14_sentence_22.wav              0
2    actor1_call_9_sentence_6.wav              0
3    actor1_call_7_sentence_5.wav              1
4  actor1_call_15_sentence_15.wav              0
